# YOLOv11 Local Person Detection

This notebook demonstrates person detection using YOLOv11 model in embedded/local mode.

## Features:
- Load YOLOv11 model
- Process sample images and videos
- Visualize detections with bounding boxes
- Calculate performance metrics

In [ ]:
import sys
sys.path.insert(0, '..')

import cv2
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import time

from src.detection.yolo_detector import YOLODetector
from src.detection.visualizer import draw_detections, create_detection_summary, draw_summary_on_frame
from src.utils.config import Config

%matplotlib inline
plt.rcParams['figure.figsize'] = (12, 8)

## 1. Download and Load YOLOv11 Model

In [ ]:
# Download YOLOv11n model (smallest, fastest)
from ultralytics import YOLO

# This will download the model if not present
model = YOLO('yolo11n.pt')
print(f"Model loaded successfully")

# Save to models directory
Config.ensure_directories()
model_path = Config.get_model_path('yolo11n.pt')
if not model_path.exists():
    import shutil
    shutil.copy('yolo11n.pt', model_path)
    print(f"Model saved to: {model_path}")

## 2. Initialize Detector

In [ ]:
# Initialize YOLODetector
detector = YOLODetector(
    model_path=str(Config.get_model_path('yolo11n.pt')),
    conf_threshold=0.25,
    device='cpu'  # Change to 'cuda' or 'mps' for GPU
)

print("Detector initialized")
print("Model info:", detector.get_model_info())

## 3. Test on Sample Image

In [ ]:
# Download a sample image with people
import urllib.request

sample_image_url = "https://ultralytics.com/images/bus.jpg"
sample_image_path = "sample_image.jpg"

urllib.request.urlretrieve(sample_image_url, sample_image_path)
print(f"Downloaded sample image to: {sample_image_path}")

In [ ]:
# Process image
start_time = time.time()
image, detections = detector.process_image(sample_image_path)
inference_time = (time.time() - start_time) * 1000

print(f"Inference time: {inference_time:.2f} ms")
print(f"Number of persons detected: {len(detections)}")

# Print detection details
for i, det in enumerate(detections):
    print(f"Person {i+1}: confidence={det.confidence:.2f}, bbox={[round(x, 1) for x in det.bbox]}")

In [ ]:
# Visualize detections
annotated_image = draw_detections(image, detections)

# Convert BGR to RGB for matplotlib
annotated_rgb = cv2.cvtColor(annotated_image, cv2.COLOR_BGR2RGB)

plt.figure(figsize=(14, 10))
plt.imshow(annotated_rgb)
plt.title(f"Detected {len(detections)} persons - Inference time: {inference_time:.2f} ms")
plt.axis('off')
plt.show()

## 4. Detection Summary Statistics

In [ ]:
# Create detection summary
summary = create_detection_summary(detections)

print("Detection Summary:")
for key, value in summary.items():
    print(f"  {key}: {value}")

# Draw summary on frame
annotated_with_summary = draw_summary_on_frame(annotated_image, summary)
summary_rgb = cv2.cvtColor(annotated_with_summary, cv2.COLOR_BGR2RGB)

plt.figure(figsize=(14, 10))
plt.imshow(summary_rgb)
plt.title("Detection with Summary Overlay")
plt.axis('off')
plt.show()

## 5. Process Video (if available)

In [ ]:
# NOTE: You need to provide a video file for this section
# You can download a sample or use your own train video

video_path = "sample_video.mp4"  # Update this path

if Path(video_path).exists():
    print(f"Processing video: {video_path}")
    
    frame_count = 0
    total_detections = 0
    total_time = 0
    
    # Process first 100 frames for demo
    for frame, detections in detector.process_video(video_path, skip_frames=2):
        if frame_count >= 100:
            break
        
        frame_count += 1
        total_detections += len(detections)
        
        # Display every 10th frame
        if frame_count % 10 == 0:
            annotated = draw_detections(frame, detections)
            summary = create_detection_summary(detections)
            annotated = draw_summary_on_frame(annotated, summary)
            
            plt.figure(figsize=(12, 8))
            plt.imshow(cv2.cvtColor(annotated, cv2.COLOR_BGR2RGB))
            plt.title(f"Frame {frame_count}: {len(detections)} persons detected")
            plt.axis('off')
            plt.show()
    
    print(f"\nProcessed {frame_count} frames")
    print(f"Total detections: {total_detections}")
    print(f"Average detections per frame: {total_detections/frame_count:.2f}")
else:
    print(f"Video file not found: {video_path}")
    print("Please provide a video file to test video processing")

## 6. Performance Benchmarking

In [ ]:
# Benchmark inference speed
num_runs = 10
times = []

print(f"Running {num_runs} inference iterations...")
for i in range(num_runs):
    start = time.time()
    detections = detector.detect_persons(image)
    elapsed = (time.time() - start) * 1000
    times.append(elapsed)

print(f"\nBenchmark Results ({num_runs} runs):")
print(f"  Average: {np.mean(times):.2f} ms")
print(f"  Min: {np.min(times):.2f} ms")
print(f"  Max: {np.max(times):.2f} ms")
print(f"  Std: {np.std(times):.2f} ms")
print(f"  FPS: {1000/np.mean(times):.2f}")

## Summary

This notebook demonstrated:
- Loading YOLOv11 model for person detection
- Processing images with bounding box visualization
- Generating detection statistics
- Video processing capabilities
- Performance benchmarking

Next: See `02_api_detection.ipynb` for REST API inference testing.